In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import os

# Suppress ROOT warnings for Sumw2
ROOT.gErrorIgnoreLevel = ROOT.kError

# ─── 0) 파일 경로 & 설정 (2024) ────────────────────────────────────────
data_files = [
    # EGamma0
    "EGamma0_Data_2024C.root",
    "EGamma0_Data_2024D.root",
    "EGamma0_Data_2024E.root",
    "EGamma0_Data_2024F.root",
    "EGamma0_Data_2024G.root",
    "EGamma0_Data_2024H.root",
    "EGamma0_Data_2024I.root",
    "EGamma0_v2_Data_2024I.root",

    # EGamma1
    "EGamma1_Data_2024C.root",
    "EGamma1_Data_2024D.root",
    "EGamma1_Data_2024E.root",
    "EGamma1_Data_2024F.root",
    "EGamma1_Data_2024G.root",
    "EGamma1_Data_2024H.root",
    "EGamma1_Data_2024I.root",
    "EGamma1_v2_Data_2024I.root",

    # Muon0
    "Muon0_Data_2024C.root",
    "Muon0_Data_2024D.root",
    "Muon0_Data_2024E.root",
    "Muon0_Data_2024F.root",
    "Muon0_Data_2024G.root",
    "Muon0_Data_2024H.root",
    "Muon0_Data_2024I.root",
    "Muon0_v2_Data_2024I.root",

    # Muon1
    "Muon1_Data_2024C.root",
    "Muon1_Data_2024D.root",
    "Muon1_Data_2024E.root",
    "Muon1_Data_2024F.root",
    "Muon1_Data_2024G.root",
    "Muon1_Data_2024H.root",
    "Muon1_Data_2024I.root",
    "Muon1_v2_Data_2024I.root",
]

mc_files = [
    "DY2E_2J_50_MC_2024.root",
    "DY2M_2J_50_MC_2024.root",
    "DY2T_2J_50_MC_2024.root",

    "s-channel_antitop_MC_2024.root",
    "s-channel_top_MC_2024.root",
    "ST_tW_antitop_di_MC_2024.root",
    "ST_tW_antitop_semi_MC_2024.root",
    "ST_tW_top_di_MC_2024.root",
    "ST_tW_top_semi_MC_2024.root",
    "t-channel_antitop_MC_2024.root",
    "t-channel_top_MC_2024.root",

    "Ttbar_di_MC_2024.root",
    "Ttbar_semi_MC_2024.root",

    "WJets_1J_MC_2024.root",
    "WJets_2J_MC_2024.root",
    "WJets_3J_MC_2024.root",
    "WJets_4J_MC_2024.root",

    "WW_MC_2024.root",
    "WWW_4F_MC_2024.root",
    "WZ_MC_2024.root",
    "WZZ_5F_MC_2024.root",
    "ZZ_22_MC_2024.root",
    "ZZ_4_MC_2024.root",
    "ZZ_semi_MC_2024.root",
    "ZZZ_5F_MC_2024.root",
]

# ─── 0.5) Cross section (placeholder 값) ────────────────────────────────
#  → 키 이름만 정확히 맞춰둠. 값은 네가 XSDB에서 찾아서 갈아끼우면 됨.
cross_sections_pb = {
    "DY2E_2J_50_MC_2024.root":      1.0,   # TODO: real xsec
    "DY2M_2J_50_MC_2024.root":      1.0,
    "DY2T_2J_50_MC_2024.root":      1.0,

    "s-channel_antitop_MC_2024.root":    1.0,
    "s-channel_top_MC_2024.root":        1.0,
    "ST_tW_antitop_di_MC_2024.root":     1.0,
    "ST_tW_antitop_semi_MC_2024.root":   1.0,
    "ST_tW_top_di_MC_2024.root":         1.0,
    "ST_tW_top_semi_MC_2024.root":       1.0,
    "t-channel_antitop_MC_2024.root":    1.0,
    "t-channel_top_MC_2024.root":        1.0,

    "Ttbar_di_MC_2024.root":       1.0,
    "Ttbar_semi_MC_2024.root":     1.0,

    "WJets_1J_MC_2024.root":       1.0,
    "WJets_2J_MC_2024.root":       1.0,
    "WJets_3J_MC_2024.root":       1.0,
    "WJets_4J_MC_2024.root":       1.0,

    "WW_MC_2024.root":             1.0,
    "WWW_4F_MC_2024.root":         1.0,
    "WZ_MC_2024.root":             1.0,
    "WZZ_5F_MC_2024.root":         1.0,
    "ZZ_22_MC_2024.root":          1.0,
    "ZZ_4_MC_2024.root":           1.0,
    "ZZ_semi_MC_2024.root":        1.0,
    "ZZZ_5F_MC_2024.root":         1.0,
}

# TODO: 실제 2024 통합 루미 값으로 바꿔야 함 (지금은 30 fb^-1 placeholder)
luminosity_pb = 30.0 * 1000.0  # pb^-1

out_dir = "compare_plots_mpl_stack_2024"
os.makedirs(out_dir, exist_ok=True)

# ─── 1) 히스토그램 이름 추출 ────────────────────────────────────────────
def get_hist_names(fn):
    f = ROOT.TFile.Open(fn)
    f.cd("plots")
    names = [k.GetName() for k in ROOT.gDirectory.GetListOfKeys()]
    f.Close()
    return names

# 첫 번째 MC 파일에서 plots 디렉토리의 히스토그램 이름 가져오기
hist_names = get_hist_names(mc_files[0])

# ─── 1.5) 이름 필터 적용 (mu_*, e_*, combine_* 만) ─────────────────────
prefixes = ("mu", "e", "combine")
hist_names = [
    h for h in hist_names
    if any(h.startswith(f"{p}_") for p in prefixes)
]

# ─── 2) 루프: 각 히스토그램별 Data/MC 누적 & 스택 플롯 ────────────────────
for hname in hist_names:
    print("▶", hname)

    # --- 2-1) Data 누적 ---
    data_h = None
    for fn in data_files:
        f = ROOT.TFile.Open(fn)
        h = f.Get(f"plots/{hname}")
        if h:
            if data_h is None:
                data_h = h.Clone("data_tmp")
                data_h.SetDirectory(0)
            else:
                data_h.Add(h)
        f.Close()

    if data_h is None:
        print("   ✖ No data, skipping")
        continue

    # --- 2-2) MC 샘플별 스케일 & 리스트화 ---
    mc_counts_list = []
    mc_labels      = []
    for fn in mc_files:
        f = ROOT.TFile.Open(fn)
        h = f.Get(f"plots/{hname}")
        if not h:
            f.Close()
            continue

        cnt   = f.Get("plots/count")
        n_evt = cnt.GetSumOfWeights() if cnt else 0
        if n_evt <= 0:
            f.Close()
            continue

        xsec  = cross_sections_pb.get(fn, 1.0)  # 값 없으면 1.0
        scale = (xsec * luminosity_pb) / n_evt

        tmp = h.Clone("tmp")
        tmp.SetDirectory(0)
        tmp.Sumw2()
        tmp.Scale(scale)

        nb  = tmp.GetNbinsX()
        arr = np.array([tmp.GetBinContent(i) for i in range(1, nb+1)])
        mc_counts_list.append(arr)
        mc_labels.append(os.path.splitext(fn)[0])
        f.Close()

    if not mc_counts_list:
        print("   ✖ No MC, skipping")
        continue

    # --- 2-3) 1D 히스토그램만 처리 ---
    if data_h.GetDimension() != 1:
        print("   ✖ Skipping non-1D histogram")
        continue

    # --- 2-4) NumPy 배열 변환 ---
    nb = data_h.GetNbinsX()
    edges = np.array(
        [data_h.GetBinLowEdge(i) for i in range(1, nb+1)]
        + [data_h.GetBinLowEdge(nb) + data_h.GetBinWidth(nb)]
    )
    dat_counts  = np.array([data_h.GetBinContent(i) for i in range(1, nb+1)])
    dat_err     = np.sqrt(dat_counts)
    bin_centers = 0.5 * (edges[:-1] + edges[1:])

    # --- 2-5) 그리기: CMS 스타일 2패널 (스택 + ratio) ---
    plt.style.use(hep.style.CMS)
    fig, (ax, axr) = plt.subplots(
        nrows=2, sharex=True,
        gridspec_kw={"height_ratios":[3,1], "hspace":0.05},
        figsize=(10, 8), dpi=150
    )

    # MC 스택
    hep.histplot(
        mc_counts_list, bins=edges,
        stack=True, histtype="fill",
        label=mc_labels, ax=ax
    )

    # Data 오버레이
    ax.errorbar(
        bin_centers, dat_counts, yerr=dat_err,
        fmt="o", color="black", label="Data"
    )
    ax.set_ylabel("Events / bin")
    hep.cms.label("Private Work", data=True, lumi=luminosity_pb/1000.0, year=2024, ax=ax)
    ax.legend(
        loc="upper right",
        prop={"size": 6},
        handletextpad=0.2,
        labelspacing=0.2,
        columnspacing=0.5,
    )

    # Ratio 패널
    mc_sum = np.sum(mc_counts_list, axis=0)
    mask   = mc_sum > 0
    ratio     = np.divide(dat_counts, mc_sum,
                          out=np.zeros_like(dat_counts), where=mask)
    ratio_err = np.divide(dat_err,    mc_sum,
                          out=np.zeros_like(dat_err),    where=mask)
    axr.errorbar(
        bin_centers[mask], ratio[mask],
        yerr=ratio_err[mask], fmt="o", color="black"
    )
    axr.axhline(1, color="gray", linestyle="--", linewidth=1)
    axr.set_ylabel("Data/MC")
    axr.set_xlabel(hname)
    axr.set_ylim(0.5, 1.5)

    # 파일 저장
    fig.savefig(f"{out_dir}/stack_{hname}.png")
    plt.close(fig)

print("✅ All stacked plots saved in", out_dir)
